# GalaxyPPG (P01) — Fitbit-style Analysis Notebook

This notebook mirrors the structure you saw in Fitbit repos:

1. Setup & Data Collection
2. Data Loading
3. Activity & Fitness (PPG-equivalent)
4. Heart Monitoring (PPG vs ECG ground truth)
5. Heart Rate Variability (HRV)
6. Reliability-first insight (SQI + abstain)
7. Export artifacts (GitHub-safe)

**Dataset mapping (your folder structure):**
- **Polar H10 (ECG reference):** `Dataset/P01/PolarH10/` → ground-truth HR/HRV
- **Empatica E4 (wrist PPG):** `Dataset/P01/E4/` → wearable PPG (BVP) + motion (ACC)

**Novel angle:** compute **SQI** (signal quality) and only trust HR/HRV when SQI is good (deployment-like behavior).

In [ ]:
# ----------------
# 0) Config
# ----------------
DATASET_DIR = "Dataset"   # points to the folder containing Meta.csv and P01..P24
SUBJECT = "P01"

# Windowing (real-time style)
FS_TARGET = None          # if None, infer from file when possible
WIN_SEC = 10.0
STEP_SEC = 2.0

# SQI threshold (quantile): lower -> stricter, higher -> more permissive
SQI_NOISY_QUANTILE = 0.35

ARTIFACTS_DIR = "artifacts"

import os, glob
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from scipy.signal import butter, filtfilt, welch

os.makedirs(ARTIFACTS_DIR, exist_ok=True)
np.random.seed(42)
plt.rcParams["figure.figsize"] = (11, 4)

print("Dataset dir:", os.path.abspath(DATASET_DIR))
print("Subject:", SUBJECT)

## 1) Setup & Data Collection

We load metadata (`Meta.csv`) and identify participant context (age, gender, stress scores, watch side).

In [ ]:
meta_path = os.path.join(DATASET_DIR, "Meta.csv")
meta = pd.read_csv(meta_path)
meta_row = meta[meta["UID"] == SUBJECT].iloc[0]
display(meta_row)

print("Age:", meta_row.get("AGE"), " Gender:", meta_row.get("GENDER"))
print("TSST:", meta_row.get("TSST"), " SSST:", meta_row.get("SSST"), " Watch side:", meta_row.get("GalaxyWatch"))

## 2) Data Loading

We load:
- **E4/BVP.csv** (wrist PPG)
- **E4/ACC.csv** (motion)
- **PolarH10/IBI.csv** (ECG-derived inter-beat intervals; best for HRV)

The loader below supports:
- Empatica-style CSV (start timestamp row + sample rate row + values)
- Standard CSV with time/value columns

In [ ]:
def _try_read_csv(path):
    # robust CSV reading
    return pd.read_csv(path, header=None)

def load_empatica_signal(path, n_axes=1):
    """
    Empatica-format often:
      row0: start_time_epoch_seconds
      row1: sampling_rate_hz
      rows2..: samples (1 col for BVP/TEMP; 3 cols for ACC)

    Also supports normal CSV with headers like time/value.
    Returns: t (seconds from start), x (np array), fs (float or None)
    """
    # First, attempt normal header read
    try:
        dfh = pd.read_csv(path)
        cols = [c.lower() for c in dfh.columns]
        # if it looks like a normal table with time & value
        time_col = None
        for c in dfh.columns:
            if str(c).lower() in ["time", "timestamp", "t", "datetime", "date"]:
                time_col = c
                break
        if time_col is not None and len(dfh.columns) >= 2:
            # pick remaining numeric cols as signal
            sig_cols = [c for c in dfh.columns if c != time_col]
            x = dfh[sig_cols].to_numpy(dtype=np.float32)
            t_raw = dfh[time_col].to_numpy()
            # Try to convert time to seconds-from-start
            try:
                t_raw = t_raw.astype(np.float64)
                t = t_raw - t_raw[0]
            except Exception:
                t = np.arange(len(x), dtype=np.float32)
            fs = None
            return t.astype(np.float32), x, fs
    except Exception:
        pass

    # Fall back to Empatica-format (no header)
    df = _try_read_csv(path)
    # Expect at least 3 rows
    if df.shape[0] < 3:
        raise ValueError(f"Unexpected format: {path}")

    start = float(df.iloc[0, 0])
    fs = float(df.iloc[1, 0])
    values = df.iloc[2:, :n_axes].to_numpy(dtype=np.float32)
    n = values.shape[0]
    t = (np.arange(n, dtype=np.float32) / fs)
    return t, values, fs

def load_ibi(path):
    """Load IBI file. Tries common formats.
    Returns: t (seconds from start), ibi_sec (float)
    """
    df = pd.read_csv(path)
    cols = [c.lower() for c in df.columns]
    # common: columns like 'timestamp' and 'ibi' or 'ibi_s'
    time_col = None
    for cand in ["time", "timestamp", "t", "date"]:
        if cand in cols:
            time_col = df.columns[cols.index(cand)]
            break
    # if no explicit time column, use first column as time
    if time_col is None:
        time_col = df.columns[0]

    # ibi column: look for 'ibi'
    ibi_col = None
    for cand in ["ibi", "ibi_s", "ibi_sec", "rr", "rr_interval", "rrinterval"]:
        if cand in cols:
            ibi_col = df.columns[cols.index(cand)]
            break
    if ibi_col is None:
        # if only 2 cols, assume second is ibi
        if len(df.columns) >= 2:
            ibi_col = df.columns[1]
        else:
            raise ValueError(f"Cannot find IBI column in {path}. Columns={df.columns.tolist()}")

    t_raw = df[time_col].to_numpy()
    ibi = df[ibi_col].to_numpy(dtype=np.float32)

    # Convert time to seconds-from-start if numeric, else index-based
    try:
        t_raw = t_raw.astype(np.float64)
        t = (t_raw - t_raw[0]).astype(np.float32)
    except Exception:
        t = np.arange(len(ibi), dtype=np.float32)

    # Convert ibi to seconds if it looks like milliseconds
    if np.nanmedian(ibi) > 5.0:  # likely ms
        ibi = ibi / 1000.0
    return t, ibi

subj_dir = os.path.join(DATASET_DIR, SUBJECT)
e4_dir = os.path.join(subj_dir, "E4")
polar_dir = os.path.join(subj_dir, "PolarH10")

bvp_path = os.path.join(e4_dir, "BVP.csv")
acc_path = os.path.join(e4_dir, "ACC.csv")
ibi_path = os.path.join(polar_dir, "IBI.csv")

assert os.path.exists(bvp_path), f"Missing: {bvp_path}"
assert os.path.exists(acc_path), f"Missing: {acc_path}"
assert os.path.exists(ibi_path), f"Missing: {ibi_path}"

t_bvp, bvp, fs_bvp = load_empatica_signal(bvp_path, n_axes=1)
t_acc, acc, fs_acc = load_empatica_signal(acc_path, n_axes=3)
t_ibi, ibi = load_ibi(ibi_path)

bvp = bvp.squeeze(-1) if bvp.ndim == 2 and bvp.shape[1] == 1 else bvp

print("BVP:", bvp.shape, " fs:", fs_bvp)
print("ACC:", acc.shape, " fs:", fs_acc)
print("IBI:", ibi.shape, " (seconds)")

### Quick visual check (raw)
We plot a short segment of wrist PPG (BVP), motion magnitude, and IBI sequence.

In [ ]:
def acc_magnitude(a):
    a = np.asarray(a, dtype=np.float32)
    if a.ndim == 2 and a.shape[1] >= 3:
        return np.sqrt((a[:,0]**2) + (a[:,1]**2) + (a[:,2]**2))
    return np.abs(a).astype(np.float32)

acc_mag = acc_magnitude(acc)

n_bvp = min(len(bvp), int((fs_bvp or 64) * 20))
plt.figure(); plt.plot(t_bvp[:n_bvp], bvp[:n_bvp]); plt.title("E4 BVP (raw) — first ~20s"); plt.xlabel("sec"); plt.show()

n_acc = min(len(acc_mag), int((fs_acc or 32) * 20))
plt.figure(); plt.plot(t_acc[:n_acc], acc_mag[:n_acc]); plt.title("E4 ACC magnitude — first ~20s"); plt.xlabel("sec"); plt.show()

plt.figure(); plt.plot(t_ibi[:min(len(ibi),200)], ibi[:min(len(ibi),200)]); plt.title("Polar IBI (sec) — first 200 intervals"); plt.xlabel("sec"); plt.show()

## 3) Activity & Fitness (PPG-equivalent)

Fitbit has steps/active minutes. Here we use wearable equivalents:
- **Motion** (ACC magnitude)
- **Signal quality (SQI)**: how concentrated the spectrum is in plausible HR band


In [ ]:
def bandpass_ppg(x, fs, low=0.7, high=4.0):
    b, a = butter(3, [low/(fs/2), high/(fs/2)], btype='band')
    return filtfilt(b, a, x).astype(np.float32)

def zscore(x, eps=1e-8):
    x = x.astype(np.float32)
    return (x - x.mean()) / (x.std() + eps)

def window_signal(x, fs, win_sec, step_sec):
    win = int(win_sec*fs)
    step = int(step_sec*fs)
    out, centers = [], []
    for s in range(0, len(x)-win+1, step):
        out.append(x[s:s+win])
        centers.append((s + win/2)/fs)
    return (np.stack(out) if out else np.zeros((0, win), dtype=np.float32)), np.array(centers, dtype=np.float32)

def sqi_power_ratio(ppg, fs):
    f, pxx = welch(ppg, fs=fs, nperseg=min(len(ppg), int(fs*4)))
    total = np.trapz(pxx, f) + 1e-12
    band = (f >= 0.7) & (f <= 4.0)
    bandp = np.trapz(pxx[band], f[band])
    return float(bandp/total)

def hr_from_psd(ppg, fs):
    f, pxx = welch(ppg, fs=fs, nperseg=min(len(ppg), int(fs*4)))
    band = (f >= 0.7) & (f <= 4.0)
    fb, pb = f[band], pxx[band]
    if len(fb) == 0:
        return float("nan")
    return float(fb[int(np.argmax(pb))] * 60.0)

# Infer FS
FS_PPG = int(round(fs_bvp)) if fs_bvp is not None else 64
FS_ACC = int(round(fs_acc)) if fs_acc is not None else 32
print("Using FS_PPG=", FS_PPG, "FS_ACC=", FS_ACC)

bvp_f = zscore(bandpass_ppg(bvp, FS_PPG))

X_ppg, t_center = window_signal(bvp_f, FS_PPG, WIN_SEC, STEP_SEC)
X_acc, _ = window_signal(acc_mag, FS_ACC, WIN_SEC, STEP_SEC)

sqi = np.array([sqi_power_ratio(w, FS_PPG) for w in X_ppg], dtype=np.float32)
motion = np.array([float(np.mean(w)) for w in X_acc], dtype=np.float32)

thr = float(np.quantile(sqi, SQI_NOISY_QUANTILE))
clean = (sqi >= thr).astype(int)

print("windows:", len(sqi), " SQI_thr:", thr, " clean%:", clean.mean()*100)

plt.figure(); plt.hist(sqi, bins=50); plt.axvline(thr); plt.title("SQI distribution"); plt.show()
plt.figure(); plt.scatter(motion, sqi, s=8); plt.title("Motion vs SQI"); plt.xlabel("ACC magnitude (mean)"); plt.ylabel("SQI"); plt.show()

## 4) Heart Monitoring (PPG vs ECG ground truth)

Ground truth is computed from **Polar IBI**:
- HR_true(window) = 60 / mean(IBI in window)

Wearable estimate from wrist PPG (BVP):
- HR_pred(window) from PSD peak

**Reliability-first:** Compare error vs SQI and show improvement when we abstain on noisy windows.

In [ ]:
def hr_from_ibi_window(t_ibi, ibi, t0, t1):
    mask = (t_ibi >= t0) & (t_ibi < t1)
    if not np.any(mask):
        return float("nan")
    m = float(np.nanmean(ibi[mask]))
    if m <= 0:
        return float("nan")
    return 60.0 / m

hr_true = []
hr_pred = []

for tc, w in zip(t_center, X_ppg):
    t0 = float(tc - WIN_SEC/2)
    t1 = float(tc + WIN_SEC/2)
    hr_true.append(hr_from_ibi_window(t_ibi, ibi, t0, t1))
    hr_pred.append(hr_from_psd(w, FS_PPG))

hr_true = np.array(hr_true, dtype=np.float32)
hr_pred = np.array(hr_pred, dtype=np.float32)

valid = np.isfinite(hr_true) & np.isfinite(hr_pred)
mae_all = np.mean(np.abs(hr_true[valid] - hr_pred[valid]))

valid_clean = valid & (clean == 1)
mae_clean = np.mean(np.abs(hr_true[valid_clean] - hr_pred[valid_clean])) if np.any(valid_clean) else float("nan")
coverage = float(np.mean(valid_clean)) * 100

print(f"HR MAE (all valid windows):   {mae_all:.2f} bpm")
print(f"HR MAE (clean only):          {mae_clean:.2f} bpm | coverage={coverage:.1f}%")

plt.figure();
plt.plot(hr_pred[:300], label='HR_pred (PPG)')
plt.plot(hr_true[:300], label='HR_true (Polar IBI)')
plt.title('Heart rate — first 300 windows')
plt.legend();
plt.show()

err = np.abs(hr_true - hr_pred)
plt.figure();
plt.scatter(sqi[valid], err[valid], s=8)
plt.title('HR absolute error vs SQI')
plt.xlabel('SQI')
plt.ylabel('|HR_true - HR_pred|')
plt.show()

## 5) Heart Rate Variability (HRV)

Compute time-domain HRV from **Polar IBI** per window:
- **RMSSD**
- **SDNN**

**Novel reliability behavior:** HRV is only interpreted on clean windows (SQI high).

In [ ]:
def hrv_rmssd_sdnn(ibi_sec):
    ibi_sec = np.asarray(ibi_sec, dtype=np.float32)
    ibi_sec = ibi_sec[np.isfinite(ibi_sec)]
    if len(ibi_sec) < 4:
        return float('nan'), float('nan')
    diff = np.diff(ibi_sec)
    rmssd = float(np.sqrt(np.mean(diff**2)))
    sdnn = float(np.std(ibi_sec))
    return rmssd, sdnn

rmssd_list, sdnn_list = [], []
for tc in t_center:
    t0 = float(tc - WIN_SEC/2)
    t1 = float(tc + WIN_SEC/2)
    mask = (t_ibi >= t0) & (t_ibi < t1)
    r, s = hrv_rmssd_sdnn(ibi[mask])
    rmssd_list.append(r)
    sdnn_list.append(s)

rmssd = np.array(rmssd_list, dtype=np.float32)
sdnn = np.array(sdnn_list, dtype=np.float32)

plt.figure(); plt.hist(rmssd[np.isfinite(rmssd)], bins=40); plt.title("RMSSD (Polar) per window"); plt.show()
plt.figure(); plt.hist(sdnn[np.isfinite(sdnn)], bins=40); plt.title("SDNN (Polar) per window"); plt.show()

# show clean-only distributions
plt.figure(); plt.hist(rmssd[(clean==1) & np.isfinite(rmssd)], bins=40); plt.title("RMSSD (Polar) — clean windows only"); plt.show()

## 6) Reliability-first summary (P01)

We export a compact, GitHub-safe table:
- window center time
- SQI
- motion
- HR_true (Polar)
- HR_pred (PPG)
- RMSSD/SDNN (Polar)


In [ ]:
out = pd.DataFrame({
    "UID": SUBJECT,
    "t_center_sec": t_center,
    "sqi": sqi,
    "clean": clean,
    "motion_mean": motion,
    "hr_true_polar": hr_true,
    "hr_pred_ppg": hr_pred,
    "hr_abs_err": np.abs(hr_true - hr_pred),
    "rmssd_polar": rmssd,
    "sdnn_polar": sdnn,
    "TSST": meta_row.get("TSST"),
    "SSST": meta_row.get("SSST"),
    "WatchSide": meta_row.get("GalaxyWatch"),
})

save_path = os.path.join(ARTIFACTS_DIR, f"{SUBJECT}_window_metrics.csv")
out.to_csv(save_path, index=False)
print("Saved:", save_path)
out.head()

## 7) Next step: scale to all subjects

Once P01 looks correct (plots, FS, file parsing), we will:

- wrap the P01 pipeline into `process_subject(uid)`
- loop over `Meta.csv[UID]` (P01–P24)
- concatenate outputs into one dataset
- add per-subject summary tables (MAE all vs clean, coverage)

✅ This is exactly how Fitbit repos scale from one user to many.